In [1]:
#conda install azure-common azure-ai-ml==0.1.0b7 mltable==0.1.0b4 azureml_dataprep azureml_dataprep_rslex responsibleai raiwidgets pandas pyarrow shap 

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import sklearn
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

#from raiwidgets import ResponsibleAIDashboard
#from responsibleai import RAIInsights
from urllib.request import urlretrieve
import zipfile

In [5]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment, BuildContext
from azureml.mlflow import register_model
import mlflow
import pandas as pd

#connect to the workspace
registry_name = "azureml"
credential = DefaultAzureCredential()
ml_client =  MLClient.from_config(credential=credential)

ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name
    )

Found the config file in: /config.json


In [6]:
compute_name = "trainingcompute"

In [7]:
from azure.ai.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if compute_name in all_compute_names:
    print(f"Found existing compute: {compute_name}")
else:
    my_compute = AmlCompute(
        name=compute_name,
        size="Standard_DS2_v2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600
    )
    ml_client.compute.begin_create_or_update(my_compute)
    print("Initiated compute creation")

Found existing compute: trainingcompute


In [8]:
rai_hospital_classifier_version_string = '141'
version='1'

In [9]:
train_data = pd.read_parquet('data/train_dataset.parquet')
test_data = pd.read_parquet('data/test_dataset.parquet')

In [10]:
target_column = "readmit_status"

In [11]:
display(train_data)

,race,gender,age,discharge_destination,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,prior_outpatient,...,prior_inpatient,primary_diagnosis,number_diagnoses,max_glu_serum,A1Cresult,insulin,diabetes_Med_prescribe,readmit_status,medicare,medicaid
3572,Caucasian,Male,Over 60 years,Other,7,12,85,2,21,0,...,0,0.0,9,None,Norm,Up,Yes,Not readmitted,False,False
543,Caucasian,Male,Over 60 years,Discharged to Home,1,3,42,0,17,0,...,1,0.0,8,None,None,Steady,Yes,Not readmitted,True,False
2074,Caucasian,Female,Over 60 years,Discharged to Home,7,7,48,1,20,0,...,0,0.0,9,None,None,No,No,Not readmitted,True,False
4190,Caucasian,Male,30-60 years,Discharged to Home,1,4,38,3,18,0,...,0,0.0,9,None,None,Down,Yes,Readmitted,False,True
42,Hispanic,Female,30-60 years,Other,7,1,50,0,8,0,...,0,0.0,9,None,>8,No,No,Not readmitted,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3249,Caucasian,Female,Over 60 years,Discharged to Home,7,4,14,0,15,0,...,0,0.0,9,None,None,No,Yes,Not readmitted,True,False
1261,AfricanAmerican,Male,Over 60 years,Discharged to Home,7,5,40,0,12,0,...,2,0.0,9,None,None,No,Yes,Readmitted,False,False
4917,Caucasian,Female,Over 60 years,Discharged to Home,7,2,37,0,15,0,...,0,0.0,8,None,None,Steady,Yes,Not readmitted,False,False
2085,Caucasian,Male,30-60 years,Other,1,4,29,2,25,0,...,0,0.0,3,None,None,No,No,Not readmitted,False,False


In [12]:
print(train_data[target_column].value_counts())

Not readmitted    3280
Readmitted         696
Name: readmit_status, dtype: int64


In [13]:
print(test_data[target_column].value_counts())

Not readmitted    811
Readmitted        183
Name: readmit_status, dtype: int64


In [14]:
train_data.head()

,race,gender,age,discharge_destination,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,prior_outpatient,...,prior_inpatient,primary_diagnosis,number_diagnoses,max_glu_serum,A1Cresult,insulin,diabetes_Med_prescribe,readmit_status,medicare,medicaid
3572,Caucasian,Male,Over 60 years,Other,7,12,85,2,21,0,...,0,0.0,9,None,Norm,Up,Yes,Not readmitted,False,False
543,Caucasian,Male,Over 60 years,Discharged to Home,1,3,42,0,17,0,...,1,0.0,8,None,None,Steady,Yes,Not readmitted,True,False
2074,Caucasian,Female,Over 60 years,Discharged to Home,7,7,48,1,20,0,...,0,0.0,9,None,None,No,No,Not readmitted,True,False
4190,Caucasian,Male,30-60 years,Discharged to Home,1,4,38,3,18,0,...,0,0.0,9,None,None,Down,Yes,Readmitted,False,True
42,Hispanic,Female,30-60 years,Other,7,1,50,0,8,0,...,0,0.0,9,None,>8,No,No,Not readmitted,False,False


In [15]:
import os
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes


training_dataset_filename = 'hospital_train_parquet'
testing_dataset_filename = 'hospital_test_parquet'


training_data = Data(
    name=training_dataset_filename,
    path='data/train_dataset.parquet',
    type=AssetTypes.URI_FILE,
    description="RAI hospital  train data",  
)

tr_data = ml_client.data.create_or_update(training_data)

testing_data = Data(
    name=testing_dataset_filename,
    path='data/test_dataset.parquet',
    type=AssetTypes.URI_FILE,
    description="RAI hospital  test data",  
)

te_data = ml_client.data.create_or_update(testing_data)


In [16]:
import os

os.makedirs('component', exist_ok=True)
os.makedirs('register_model_src', exist_ok=True)
os.makedirs('environment', exist_ok=True)

In [17]:
%%writefile component/hospital_training.py


from pathlib import Path
import sys
import os

parent_dir =  os.path.dirname(os.getcwd())
 
# setting path
sys.path.append(parent_dir)

import argparse
import os
import shutil
import tempfile

from azureml.core import Run

import mlflow
import mlflow.sklearn

import pandas as pd
import numpy as np
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split



def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", type=str, help="Path to training data")
    parser.add_argument("--target_column_name", type=str, help="Name of target column")
    parser.add_argument("--model_output", type=str, help="Path of output model")

    # parse args
    args = parser.parse_args()    

    # return args
    return args

def get_categorical_index(categorical_fields):
    cat_idx = []
    for col, value in categorical_fields.iteritems():
        if value.dtype == 'object':
            cat_idx.append(categorical_fields.columns.get_loc(col))
    print("col indices: ", cat_idx)  
    return cat_idx    



def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    print("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)

    # Read in data
    print("Reading data")
    all_training_data = pd.read_parquet(args.training_data)
    target = all_training_data[args.target_column_name]
    features = all_training_data.drop([args.target_column_name], axis = 1)  

    #features['age'] = pd.Categorical(all_training_data['age'], categories=['30 years or younger', '30-60 years', 'Over 60 years'], ordered=True)

    # Transform string data to numeric
    numerical_selector = selector(dtype_include=np.number)
    categorical_selector = selector(dtype_exclude=np.number)

    numerical_columns = numerical_selector(features)
    categorical_columns = categorical_selector(features)

    categorial_encoder = OneHotEncoder(handle_unknown="ignore")
    numerical_encoder = StandardScaler()

    preprocessor = ColumnTransformer([
    ('categorical-encoder', categorial_encoder, categorical_columns),
    ('standard_scaler', numerical_encoder, numerical_columns)])

    categorical_indices = get_categorical_index(features)
    #clf = make_pipeline(preprocessor, RandomForestClassifier())
    clf = make_pipeline(preprocessor, LogisticRegression())
    

    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=1)

    print("Training model...") 
    
    model = clf.fit(X_train, y_train)
 
    # Saving model with mlflow - leave this section unchanged
    model_dir =  "./model_output"
    with tempfile.TemporaryDirectory() as td:
        print("Saving model with MLFlow to temporary directory")
        tmp_output_dir = os.path.join(td, model_dir)
        mlflow.sklearn.save_model(sk_model=model, path=tmp_output_dir)

        print("Copying MLFlow model to output path")
        for file_name in os.listdir(tmp_output_dir):
            print("  Copying: ", file_name)
            # As of Python 3.8, copytree will acquire dirs_exist_ok as
            # an option, removing the need for listdir
            shutil.copy2(src=os.path.join(tmp_output_dir, file_name), dst=os.path.join(args.model_output, file_name))


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Overwriting component/hospital_training.py


In [18]:
%%writefile register_model_src/model_register.py

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------

import argparse
import json
import os
import time


from azureml.core import Run

import mlflow
import mlflow.sklearn

# Based on example:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-cli
# which references
# https://github.com/Azure/azureml-examples/tree/main/cli/jobs/train/lightgbm/iris


def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--model_input_path", type=str, help="Path to input model")
    parser.add_argument(
        "--model_info_output_path", type=str, help="Path to write model info JSON"
    )
    parser.add_argument(
        "--model_base_name", type=str, help="Name of the registered model"
    )
    parser.add_argument(
        "--model_name_suffix", type=int, help="Set negative to use epoch_secs"
    )

    # parse args
    args = parser.parse_args()

    # return args
    return args


def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    print("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)

    print("Loading model")
    mlflow_model = mlflow.sklearn.load_model(args.model_input_path)

    if args.model_name_suffix < 0:
        suffix = int(time.time())
    else:
        suffix = args.model_name_suffix
    registered_name = "{0}_{1}".format(args.model_base_name, suffix)
    print(f"Registering model as {registered_name}")

    print("Registering via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=mlflow_model,
        registered_model_name=registered_name,
        artifact_path=registered_name,
    )

    print("Writing JSON")
    dict = {"id": "{0}:1".format(registered_name)}
    output_path = os.path.join(args.model_info_output_path, "model_info.json")
    with open(output_path, "w") as of:
        json.dump(dict, fp=of)


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Overwriting register_model_src/model_register.py


In [19]:
from azure.ai.ml import load_component

yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: rai_hospital_training_component
display_name: hospital  classification training component for RAI example
version: {rai_hospital_classifier_version_string}
type: command
inputs:
  training_data:
    type: path
  target_column_name:
    type: string
outputs:
  model_output:
    type: path
code: ./component/
environment: azureml://registries/azureml/environments/AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu/versions/4
""" + r"""
command: >-
  python hospital_training.py
  --training_data ${{{{inputs.training_data}}}}
  --target_column_name ${{{{inputs.target_column_name}}}}
  --model_output ${{{{outputs.model_output}}}}
"""

yaml_filename = "RAIhospitalClassificationTrainingComponent.yaml"

with open(yaml_filename, 'w') as f:
    f.write(yaml_contents.format(yaml_contents))
    
train_component_definition = load_component(
    source=yaml_filename
)

ml_client.components.create_or_update(train_component_definition)

CommandComponent({'intellectual_property': None, 'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.COMPONENT', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'rai_hospital_training_component', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/8a0f6419-1f4c-45b3-8d92-ee53be1ea443/resourceGroups/demorg/providers/Microsoft.MachineLearningServices/workspaces/aml-ws/components/rai_hospital_training_component/versions/141', 'Resource__source_path': None, 'base_path': PosixPath('.'), 'creation_context': <azure.ai.ml._restclient.v2022_10_01.models._models_py3.SystemData object at 0x7f8cd2600760>, 'serialize': <msrest.serialization.Serializer object at 0x7f8cd2599310>, 'command': 'python hospital_training.py --training_data ${{inputs.training_data}} --target_column_name ${{inputs.target_column_name}} --model_output ${{outputs.model_output}}', 'code': '/subscriptions/8a0f6419-1f4c-45b3-8d92-ee53be1ea443/resourceGroups/demorg/p

In [20]:
yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: register_hospital_model
display_name: Register hospital Model
version: {rai_hospital_classifier_version_string}
type: command
is_deterministic: False
inputs:
  model_input_path:
    type: path
  model_base_name:
    type: string
  model_name_suffix: # Set negative to use epoch_secs
    type: integer
    default: -1
outputs:
  model_info_output_path:
    type: path
code: ./register_model_src/
environment: azureml://registries/azureml/environments/AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu/versions/4
command: >-
  python model_register.py
  --model_input_path ${{{{inputs.model_input_path}}}}
  --model_base_name ${{{{inputs.model_base_name}}}}
  --model_name_suffix ${{{{inputs.model_name_suffix}}}}
  --model_info_output_path ${{{{outputs.model_info_output_path}}}}

"""
yaml_filename = "model_register.yaml"

with open(yaml_filename, 'w') as f:
    f.write(yaml_contents)
    
register_component = load_component(
    source=yaml_filename
)

ml_client.components.create_or_update(register_component)

CommandComponent({'intellectual_property': None, 'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.COMPONENT', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'register_hospital_model', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/8a0f6419-1f4c-45b3-8d92-ee53be1ea443/resourceGroups/demorg/providers/Microsoft.MachineLearningServices/workspaces/aml-ws/components/register_hospital_model/versions/141', 'Resource__source_path': None, 'base_path': PosixPath('.'), 'creation_context': <azure.ai.ml._restclient.v2022_10_01.models._models_py3.SystemData object at 0x7f8cd258ec40>, 'serialize': <msrest.serialization.Serializer object at 0x7f8cd25d8eb0>, 'command': 'python model_register.py --model_input_path ${{inputs.model_input_path}} --model_base_name ${{inputs.model_base_name}} --model_name_suffix ${{inputs.model_name_suffix}} --model_info_output_path ${{outputs.model_info_output_path}}', 'code': '/subscriptions/8a0f6419-

In [21]:
import time

model_name_suffix = int(time.time())
model_base_name = 'rai_hospital_model'

In [22]:
from azure.ai.ml import dsl, Input


hospital_train_parquet = Input(
    type="uri_file", path="data/train_dataset.parquet", mode="download"
)

hospital_test_parquet = Input(
    type="uri_file", path="data/test_dataset.parquet", mode="download"
)

@dsl.pipeline(
    compute=compute_name,
    description="Register Model for RAI hospital ",
    experiment_name=f"RAI_hospital_Model_Training_{model_name_suffix}",
)
def my_training_pipeline(target_column_name, training_data):
    trained_model = train_component_definition(
        target_column_name=target_column_name,
        training_data=training_data
    )
    trained_model.set_limits(timeout=120)

    _ = register_component(
        model_input_path=trained_model.outputs.model_output,
        model_base_name=model_base_name,
        model_name_suffix=model_name_suffix,
    )

    return {}

model_registration_pipeline_job = my_training_pipeline(target_column, hospital_train_parquet)

In [23]:
from azure.ai.ml.entities import PipelineJob
import webbrowser

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    while created_job.status not in ['Completed', 'Failed', 'Canceled', 'NotResponding']:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))


    # open the pipeline in web browser
    webbrowser.open(created_job.services["Studio"].endpoint)
    
    #assert created_job.status == 'Completed'
    return created_job

# This is the actual submission
training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Completed


In [24]:
expected_model_id = f'{model_base_name}_{model_name_suffix}:1'
azureml_model_id = f'azureml:{expected_model_id}'

In [25]:
def get_categorical_numerical_data(dataset):
    dataset = dataset.drop([target_column], axis = 1)  
    categorical = []
    for col, value in dataset.iteritems():
        if value.dtype == 'object' or value.dtype == 'bool':
            categorical.append(col)
    numerical = dataset.columns.difference(categorical)
    return categorical, numerical

In [26]:
# get categorical and numerical fields from training data
categorical, numerical = get_categorical_numerical_data(train_data)
print("categorical columns: ",  categorical)
print("numerical field: ", numerical)

categorical columns:  ['race', 'gender', 'age', 'discharge_destination', 'max_glu_serum', 'A1Cresult', 'insulin', 'diabetes_Med_prescribe', 'medicare', 'medicaid']
numerical field:  Index(['admission_source', 'num_lab_procedures', 'num_medications',
       'num_procedures', 'number_diagnoses', 'primary_diagnosis',
       'prior_emergency', 'prior_inpatient', 'prior_outpatient',
       'time_in_hospital'],
      dtype='object')


In [27]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# We get latest version and use the same version for all components
version = rai_constructor_component.version

rai_counterfactual_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_counterfactual", version=version
)

rai_causal_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_causal", version=version
)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

rai_scorecard_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_score_card", version=version
)

In [28]:
import json

score_card_config_dict = {
    "Model": {
        "ModelName": "Diabetes hospital readmission",
        "ModelType": "Classification",
        "ModelSummary": "This model predicts whether a diabetic patient will be readmitted back to hospital within 30 days"
    },
    "Metrics" :{
        "accuracy_score": {
            "threshold": ">=0.80"
        },
        "precision_score": {}
    },
    "FeatureImportance": { 
        "top_n": 10 
    }, 
    "DataExplorer": { 
        "features": [ 
        "time_in_hospital", 
        "prior_inpatient"
        ] 
    }
}

score_card_config_filename = "rai_hospital_readmission_score_card_config.json"

with open(score_card_config_filename, 'w') as f:
    json.dump(score_card_config_dict, f)

In [29]:
import json

score_card_config_path = Input(
    type="uri_file",
    path=score_card_config_filename,
    mode="download"
)

@dsl.pipeline(
        compute=compute_name,
        description="RAI computation on hospital readmit classification data",
        experiment_name=f"RAI_hospital_Classification_RAIInsights_Computation_{model_name_suffix}",
    )
def rai_classification_pipeline(
        target_column_name,
        training_data,
        testing_data,
        score_card_config_path
    ):
        
        # Initiate the RAIInsights
        create_rai_job = rai_constructor_component(
            title="RAI Dashboard",
            task_type="classification",
            model_info=expected_model_id,
            model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),            
            train_dataset=training_data,
            test_dataset=testing_data,
            target_column_name=target_column_name,
            #classes=json.dumps(['not readmitted', 'readmitted']),
            categorical_column_names=json.dumps(categorical),
        )
        create_rai_job.set_limits(timeout=120)
        
        # Add an explanation
        explain_job = rai_explanation_component(
            comment="Explanation for hospital remitted less than 30days  classification",
            rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        )
        explain_job.set_limits(timeout=120)
        
        # Add error analysis
        erroranalysis_job = rai_erroranalysis_component(
            rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        )
        erroranalysis_job.set_limits(timeout=120)

        # Add counterfactual analysis
        counterfactual_job = rai_counterfactual_component(
            rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
            total_cfs=10,
            desired_class='opposite',
        )
        counterfactual_job.set_limits(timeout=600)

        # Add causal analysis
        causal_job = rai_causal_component(
            treatment_features=json.dumps(['time_in_hospital']),
            rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        )
        causal_job.set_limits(timeout=120)
        

        # Combine everything
        rai_gather_job = rai_gather_component(
            constructor=create_rai_job.outputs.rai_insights_dashboard,
            insight_1=explain_job.outputs.explanation,
            insight_2=causal_job.outputs.causal,
            insight_3=counterfactual_job.outputs.counterfactual,
            insight_4=erroranalysis_job.outputs.error_analysis,
        )
        rai_gather_job.set_limits(timeout=120)

        rai_gather_job.outputs.dashboard.mode = "upload"
        rai_gather_job.outputs.ux_json.mode = "upload"

        # Generate score card in pdf format for a summary report on model performance,
        # and observe distrbution of error between prediction vs ground truth.
        rai_scorecard_job = rai_scorecard_component(
            dashboard=rai_gather_job.outputs.dashboard,
            pdf_generation_config=score_card_config_path
        )

        return {
            "dashboard": rai_gather_job.outputs.dashboard,
            "ux_json": rai_gather_job.outputs.ux_json,
            "scorecard": rai_scorecard_job.outputs.scorecard
        }

In [30]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_classification_pipeline(
    target_column_name=target_column,
    training_data=hospital_train_parquet,
    testing_data=hospital_test_parquet,
    score_card_config_path=score_card_config_path,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)
insights_pipeline_job.outputs.ux_json = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/ux_json/",
    mode="upload",
    type="uri_folder",
)
insights_pipeline_job.outputs.scorecard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/scorecard/",
    mode="upload",
    type="uri_folder",
)

# submit pipeline
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

Uploading rai_hospital_readmission_score_card_config.json (< 1 MB): 100%|██████████| 391/391 [00:00<00:00, 28.3kB/s]




Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Completed


In [ ]:
sub_id = ml_client._operation_scope.subscription_id
rg_name = ml_client._operation_scope.resource_group_name
ws_name = ml_client.workspace_name

expected_uri = f"https://ml.azure.com/model/{expected_model_id}/model_analysis?wsid=/subscriptions/{sub_id}/resourcegroups/{rg_name}/workspaces/{ws_name}"

print(f"Please visit {expected_uri} to see your analysis")